In [1]:
import urllib
from urllib2 import urlopen
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bs4 import BeautifulSoup
from collections import defaultdict
from bson.objectid import ObjectId
%matplotlib inline

db_client = MongoClient()
data = db_client['shopify']['forum']
data2 = db_client['shopify_backup']
users = db_client['shopify']['users']
post_db = db_client['shopify']['posts']
shopify_post_db = db_client['shopify']['shopify_posts']

In [2]:
base_url = 'https://ecommerce.shopify.com/'
forums_url = 'forums'

<div style="text-align:center;padding:10px;"><h1>Step 3: Scrape User and Topic Information</h1></div>

In [6]:
for category in [data.find({'category':'category-shopify-discussion'})[0]]:
    
    print '-----> CATEGORY:', category['category']
    for sub_cat_key in category["sub_cats"]:
        
        if sub_cat_key == 'category-ecommerce-design':
            
            print 'SUBCATEGORY: {sub_cat}'.format(sub_cat=sub_cat_key)


            sub_cat = category['sub_cats'][sub_cat_key]
            posts = []


            # Run through each thread
            for idx, url in enumerate(sub_cat['topic_urls']):

                if idx > 17500:
                    try:
                        topic_page = BeautifulSoup(urlopen(url))
                        thread = []

                        # Process original post
                        user_id, text, date = process_post(topic_page.find('div',{'class':'original-post'}))
                        shopify_post_db.insert_one({'user_id':user_id, 'date':date, 'text':text})

                        # Process each reply
                        for reply in topic_page.find_all('div',{'itemtype':'http://schema.org/UserComments'}):
                            user_id, text, date = process_post(reply)
                            shopify_post_db.insert_one({'user_id':user_id, 'date':date, 'text':text})


                        # Status Update
                        if (idx+1) % 5 == 0:
                            print 'PROCESSED {idx}/{total} posts'.format(idx=(idx+1),total=len(sub_cat['topic_urls']))

                    except Exception as e:
                        print 'ERROR IN SUBCATEGORY: ', str(e)

-----> CATEGORY: category-shopify-discussion
SUBCATEGORY: category-ecommerce-design
PROCESSED 17505/20620 posts
PROCESSED 17510/20620 posts
PROCESSED 17515/20620 posts
PROCESSED 17520/20620 posts
PROCESSED 17525/20620 posts
PROCESSED 17530/20620 posts
PROCESSED 17535/20620 posts
PROCESSED 17540/20620 posts
PROCESSED 17545/20620 posts
PROCESSED 17550/20620 posts
PROCESSED 17555/20620 posts
PROCESSED 17560/20620 posts
PROCESSED 17565/20620 posts
PROCESSED 17570/20620 posts
PROCESSED 17575/20620 posts
PROCESSED 17580/20620 posts
PROCESSED 17585/20620 posts
PROCESSED 17590/20620 posts
PROCESSED 17595/20620 posts
PROCESSED 17600/20620 posts
PROCESSED 17605/20620 posts
PROCESSED 17610/20620 posts
PROCESSED 17615/20620 posts
PROCESSED 17620/20620 posts
PROCESSED 17625/20620 posts
PROCESSED 17630/20620 posts
PROCESSED 17635/20620 posts
PROCESSED 17640/20620 posts
PROCESSED 17645/20620 posts
PROCESSED 17650/20620 posts
PROCESSED 17655/20620 posts
PROCESSED 17660/20620 posts
PROCESSED 17665/2062

In [3]:
def process_post(post):
    
    user = post.find('div',{'class':'author'})
    user_id = int(user.find('a',{'class':'username'}).get('href').split('/')[-1])
    
    # Add user to db if does not exist 
    if users.find({'id': user_id}).count() < 1:
        name = user.find('a',{'class':'username'}).text.strip()
        title = user.find_all('em')[0].text.strip().lower()
        site = user.find_all('em')[-1].text.strip().lower()
        posts = int(user.find('div',{'class':'stats'}).find('a').text.strip().lower())
        users.insert_one({'id':user_id,'name':name,'title':title,'site':site, 'posts':posts })
        
    # Extract date
    date = get_date(post)
    
    # Extract text
    text = ' '.join([txt.text.lower() for txt in post.find('div',{'itemprop':'description'}).find_all('p')])
    
    return user_id, text, date
    
    


In [4]:
import re
import datetime as dt
months = ['january','february','march','april','may','june','july','august','september','october','november','december']

def get_date(post):
    
    regexes = ['(\d+) day','(\d+) month', '(\w+) (\d+), (20\d+)']
    raw_date = post.find('div',{'class':'date'}).text.strip()
    date = None
    
    for idx, reg in enumerate(regexes):
        matches = re.search(reg, raw_date)
        if matches:
            if idx == 0:
                date = dt.datetime.now() - dt.timedelta(days=int(matches.group(1)))
            elif idx == 1:
                date = dt.datetime.now() - dt.timedelta(days=int(matches.group(1))*30)
            else:
                date = dt.datetime(int(matches.group(3)), int(months.index(matches.group(1).lower())) + 1, int(matches.group(2)))

    return date.strftime('%Y-%m-%d') if date else 'WHY'